In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep

In [30]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDUwNDE1NTMwMDExMTEyNzc='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 8월 2일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):     
        address_no_zero = address
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=5' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        sleep(1)
        roadname_info = roadname_result#.json()['results']['juso']        
        if roadname_info == []:
            print(address_no_zero)
            continue
                                
        roadname_info = roadname_info[0]               
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        sleep(1)
        #return coor_result
        
        if coor_result.json()['results']['juso'] == None:
            print(address_no_zero)
            continue
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [3]:
basedir = './지하철역/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.xlsx')]
filenames

['서울교통공사_역주소 및 전화번호_20200715.xlsx']

In [4]:
subway_df = pd.read_excel(basedir + filenames[0])
print(subway_df.shape)
subway_df.head()

(292, 6)


,연번,역번호,호선,역명,역전화번호,역주소
0,1,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동)
2,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가)
3,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가)
4,5,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가)


In [8]:
subway_df['역주소']

0                서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1                   서울특별시 중구 세종대로 지하 101 (정동)
2                   서울특별시 종로구 종로 지하 55 (종로1가)
3                  서울특별시 종로구 종로 지하 129 (종로3가)
4                 서울특별시 종로구 종로 지하 216  (종로5가)
5                   서울특별시 종로구 종로 지하 302 (창신동)
6                   서울특별시 동대문구 왕산로 지하 1 (신설동)
7                  서울특별시 동대문구 왕산로 지하 93 (제기동)
8                  서울특별시 동대문구 왕산로 지하 205(전농동)
9                      서울특별시 종로구 종로 359 (숭인동)
10                   서울특별시 강서구 강서로 지하54 (화곡동)
11                  서울특별시 중구 세종대로 지하 101 (정동)
12                 서울특별시 중구 을지로 지하 42 (을지로1가)
13                서울특별시 중구 을지로 지하 106 (을지로3가)
14                서울특별시 중구 을지로 지하 178 (을지로4가)
15                서울특별시 중구 을지로 지하 279 (을지로7가)
16                서울특별시 중구 퇴계로 지하 431-1 (신당동)
17              서울특별시 성동구 왕십리로 지하 374 (하왕십리동)
18                 서울특별시 성동구 왕십리로 지하300 (행당동)
19                   서울특별시 성동구 왕십리로 206 (행당동)
20                  서울특별시 성동구 아차산로 18 (성수동1가)
21                 서울특별시 성동구 아차산로 

In [7]:
subway_df['역주소'].apply(lambda x: x.split('(')[0])

0           서울특별시 중구 세종대로 지하 2 
1         서울특별시 중구 세종대로 지하 101 
2           서울특별시 종로구 종로 지하 55 
3          서울특별시 종로구 종로 지하 129 
4         서울특별시 종로구 종로 지하 216  
5          서울특별시 종로구 종로 지하 302 
6          서울특별시 동대문구 왕산로 지하 1 
7         서울특별시 동대문구 왕산로 지하 93 
8         서울특별시 동대문구 왕산로 지하 205
9             서울특별시 종로구 종로 359 
10          서울특별시 강서구 강서로 지하54 
11        서울특별시 중구 세종대로 지하 101 
12          서울특별시 중구 을지로 지하 42 
13         서울특별시 중구 을지로 지하 106 
14         서울특별시 중구 을지로 지하 178 
15         서울특별시 중구 을지로 지하 279 
16       서울특별시 중구 퇴계로 지하 431-1 
17       서울특별시 성동구 왕십리로 지하 374 
18        서울특별시 성동구 왕십리로 지하300 
19          서울특별시 성동구 왕십리로 206 
20           서울특별시 성동구 아차산로 18 
21          서울특별시 성동구 아차산로 100 
22          서울특별시 광진구 아차산로 243 
23        서울특별시 광진구 아차산로 384-1 
24           서울특별시 광진구 강변역로 53 
25            서울특별시 송파구 오금로 20 
26       서울특별시 송파구 올림픽로 지하 265 
27       서울특별시 송파구 올림픽로 지하 140 
28        서울특별시 송파구 올림픽로 지하 23 
29       서울특별시 강남구 테헤란로 지하 538 
30       서울특별시 강남구 테헤란로 지하 340 
31      

In [31]:
prac_result = fetch_coor_data(['대림역'])

0it [00:01, ?it/s]


In [32]:
prac_result.json()

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '5',
   'totalCount': '7',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '268, Gamasan-ro, Guro-gu, Seoul',
    'rn': '가마산로',
    'emdNm': '구로동',
    'zipNo': '08302',
    'roadAddrPart2': ' (구로동)',
    'emdNo': '01',
    'sggNm': '구로구',
    'jibunAddr': '서울특별시 구로구 구로동 98-4 대림역와이즈플레이스',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 구로구 가마산로 268',
    'bdNm': '대림역와이즈플레이스',
    'admCd': '1153010200',
    'udrtYn': '0',
    'lnbrMnnm': '98',
    'roadAddr': '서울특별시 구로구 가마산로 268 (구로동)',
    'lnbrSlno': '4',
    'buldMnnm': '268',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '115303000023',
    'mtYn': '0',
    'bdMgtSn': '1153010200100980004016223',
    'buldSlno': '0'},
   {'detBdNmList': '',
    'engAddr': '27, Gongwon-ro, Guro-gu, Seoul',
    'rn': '공원로',
    'emdNm': '구로동',
    'zipNo': '08298',
    'roadAddrPart2': ' (구로동, 대림역대성스카이렉스)',
    'emdNo': '01',
    'sggNm': '

In [9]:
subway_coor_df = fetch_coor_data(subway_df['역주소'].tolist())

44it [01:42,  2.10s/it]

서울 구로구 구로6동 125-66


179it [07:22,  2.17s/it]

서울특별시 은평구 증산로 지하400 (신사동


196it [08:03,  2.08s/it]

...

223it [09:09,  2.07s/it]

서울특별시 중랑구 망우로 지하 297


291it [12:01,  2.06s/it]

서울특별시 송파구 강동대로 327


292it [12:03,  2.48s/it]


In [10]:
print(subway_coor_df.shape)
subway_coor_df.head()

(287, 4)


,지번주소,도로명주소,x좌표,y좌표
0,서울특별시 중구 세종대로 지하 2 (남대문로 5가),서울특별시 중구 세종대로 지하 2,953414.452778,1.950998e+06
1,서울특별시 중구 세종대로 지하 101 (정동),서울특별시 중구 세종대로 지하 101,953808.786159,1.951915e+06
2,서울특별시 종로구 종로 지하 55 (종로1가),서울특별시 종로구 종로 지하 55,954366.404325,1.952442e+06
3,서울특별시 종로구 종로 지하 129 (종로3가),서울특별시 종로구 종로 지하 129,955141.774693,1.952461e+06
4,서울특별시 종로구 종로 지하 216 (종로5가),서울특별시 종로구 종로 지하 216,956016.193934,1.952509e+06


In [11]:
subway_coor_df.to_csv('./지하철역/subway_coor_ver_2.csv', index=False)

In [12]:
subway_coor_df2 = pd.read_csv(basedir + 'subway_coor.csv')
print(subway_coor_df2.shape)

(288, 4)


In [16]:
for i, v in enumerate(subway_df['역주소'].apply(lambda x: x.split('(')[0])):
    if v not in subway_coor_df2['지번주소'].tolist():
        print(subway_df['역명'].loc[i])
        print(subway_df['역주소'].loc[i])
        print(v)

대림
서울 구로구 구로6동 125-66
서울 구로구 구로6동 125-66
약수
서울특별시 중구 다산로 지하 155 (신당동)
서울특별시 중구 다산로 지하 155 
상봉
서울특별시 중랑구 망우로 지하 297
서울특별시 중랑구 망우로 지하 297
둔촌오륜
서울특별시 송파구 강동대로 327
서울특별시 송파구 강동대로 327
